In [169]:
import pandas as pd
import us
import glob

from tqdm import tqdm

In [170]:
birth_2022 = pd.read_csv('data_by_state/births_2022.txt',sep='\t')

In [171]:
birth_2022 = birth_2022.drop(columns=['Notes',"Year Code","Mother\'s Single Race Code"])
birth_2022 = birth_2022.drop(columns=['Gender'])
birth_2022 = birth_2022.rename(columns={"Gender Code":"Gender",'Mother\'s Single Race':"Race"})


In [172]:
birth_2022 = birth_2022.query('State!="District of Columbia"')
def get_state_abbr(state):
    res = us.states.lookup(state)
    if res is None:
        return state
    return res.abbr
birth_2022['State'] = birth_2022['State'].apply(get_state_abbr)

In [173]:
names_files = glob.glob('data_by_state/names_by_state/*.TXT',recursive=True)
names_2022 = pd.concat([pd.read_csv(f,header=None, names=['State',"Gender","Year","Name","Count"]).query('Year==2022') for f in names_files])

names_no_state = names_2022.query('State==""')


In [174]:
# fill in names with 0
all_names = names_2022['Name'].unique()
all_states = names_2022['State'].unique()

new_rows = []

with tqdm(total=len(all_states)*2) as pbar:
    for state in all_states:
        for gender in ['M','F']:
            subdf = names_2022[(names_2022.Gender==gender) & (names_2022.State==state)]
            missing_names = list(set(all_names) - set(subdf['Name']))
            for name in missing_names:
                new_rows.append({'State':state,"Gender":gender,"Year":2022,"Name":name,"Count":0})   

            pbar.update(1)

names_2022 = pd.concat([names_2022,pd.DataFrame(new_rows)])



100%|██████████| 102/102 [00:00<00:00, 140.52it/s]


In [175]:
master_df = birth_2022.merge(names_2022,on=['State','Year','Gender'])
master_df.to_csv('names_and_births_2022.csv',index=False)